### <i>Pre-processing & splitting normal and augmented data</i>

In [99]:
import os
import librosa
import torchaudio
import torch
import math
import pandas as pd
import numpy as np
from IPython.display import Audio
train_dirs = [f"../data/Train/{path}" for path in os.listdir("../data/Train")]
train_dirs+= [f"augmented/{path}" for path in os.listdir("augmented")]
train_dirs.sort()

labels_csv_out = ""
for dir in train_dirs:
    audio, sr = torchaudio.load(dir)
    file_name = dir.split("/")[-1].replace(".wav",".npy")
    label = file_name[0]
    labels_csv_out += ",".join([file_name,label])+"\n"
    length = len(audio[0])/sr
    target_length = 13 * sr
    audio = (librosa.effects.trim(audio, top_db = 40)[0]/torch.max(torch.abs(audio))).repeat(1,13//math.floor(length)+30)[0][:target_length]
    S = librosa.feature.melspectrogram(y=audio.numpy(), sr=sr)
    # print(audio.shape, S.shape)
    np.save(f'preprocessed/all/specs/{file_name}', S)
    # break
with open("preprocessed/all/labels.csv", "w+") as f:
    f.write(labels_csv_out)

In [100]:
from sklearn.model_selection import train_test_split
import shutil

# Split data
train_files = [dir.replace(".wav","") for dir in os.listdir("../data/Train")]
train_files.sort()

stratify = [name.split("_")[0] for name in train_files]

train_ids, val_ids = train_test_split(train_files, test_size=0.2, random_state=42, stratify=stratify)

for file_name in os.listdir("preprocessed/all/specs"):
    id = file_name[:7]
    src = f"preprocessed/all/specs/{file_name}"
    dst = "preprocessed/train/specs" if (id in train_ids) else "preprocessed/val/specs"
    shutil.copy(src,dst)

# Generate labels
for set_type in ["train","val"]:
    labels = ""
    for file_name in os.listdir(f"preprocessed/{set_type}/specs"):
        labels+= f"{file_name},{file_name[0]}\n"
    with open(f"preprocessed/{set_type}/labels.csv", "w+") as f:
        f.write(labels)

## Load train data

In [47]:
import convenience
df_normal_train_val, sample_rates = convenience.load_train()
file_names = df_normal_train_val.file_name
print(df_normal_train_val.head())
file_names.head()

KeyboardInterrupt: 

In [ ]:
# df_augment_train_val, sample_rates = convenience.load_train("augmented")
# df_augment_train_val.head()
df_train_val = df_normal_train_val

### Combine augmented and non-augmented data togehter

In [ ]:
import pandas as pd
# df_train_val = pd.concat([df_augment_train_val, df_normal_train_val], axis = 0, ignore_index=True)

In [ ]:
# for x in range(0,len(df_train_val),1000):
#     subset = df_train_val.loc[x:x+999]
#     subset.audio = [audio.tolist() for audio in subset.audio.tolist()]
#     display(subset)
#     subset.to_csv(f"subsets/subset_{x}:{x+999}.csv")
#     break


/tmp/ipykernel_4380/373078797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.audio = [audio.tolist() for audio in subset.audio.tolist()]


,file_name,stratify,accent,gender,audio,length
0,4m_6328_5703.wav,4m,4,m,"[-0.00152587890625, 0.000152587890625, 0.00143...",4.778688
1,1m_5200_619.wav,1m,1,m,"[-0.00146484375, -6.103515625e-05, 0.000732421...",3.584000
2,1f_2592_102.wav,1f,1,f,"[9.1552734375e-05, -0.000457763671875, -0.0007...",3.072000
3,4f_5694_5281.wav,4f,4,f,"[-0.001983642578125, -0.000213623046875, 0.002...",5.632000
4,1f_6761_333.wav,1f,1,f,"[-0.00152587890625, -9.1552734375e-05, 0.00061...",6.570625
...,...,...,...,...,...,...
995,4f_8637_5402.wav,4f,4,f,"[0.0001220703125, -0.000396728515625, -0.00045...",7.338687
996,1m_8517_3873.wav,1m,1,m,"[0.00103759765625, -0.000244140625, -0.0007324...",6.997312
997,1f_6989_3514.wav,1f,1,f,"[0.00372314453125, 0.00201416015625, 0.0024719...",4.778688
998,5f_8225_2865.wav,5f,5,f,"[-0.000579833984375, 0.000396728515625, -9.155...",7.765313


In [ ]:
import ast
from IPython.display import Audio
my_favourite_df = pd.read_csv("subsets/subset_0:999.csv")
Audio(my_favourite_df.audio[0],rate=16000)


TypeError: dtype '<class 'list'>' not understood

In [ ]:
import gc
gc.collect()

0

## Trim silence

In [ ]:
#silence cutting
import librosa 
trimmed_audio = []

for file in df_train_val.audio:
    trimmed, index = librosa.effects.trim(file, top_db = 40)    
    trimmed_audio.append(trimmed)
    
df_train_val.audio = trimmed_audio
#df_train_val["trimmed"] = trimmed_audio

In [ ]:
# df_train_val["trimmed"] = trimmed_audio
from IPython.display import Audio
Audio(df_train_val.audio[0],rate = 16000)
#df_train_val.audio[3164]

In [ ]:
df_train_val["length"] = [len(df_train_val.audio[i])/16000 for i  in range(0, len(df_train_val.audio))]

In [ ]:
import IPython.display as ipd
display(df_train_val)
display(ipd.Audio(df_train_val.audio[0], rate=16000))
display(ipd.Audio(trimmed_audio[0], rate=16000))

,file_name,stratify,accent,gender,audio,length
0,5f_1561.wav,5f,5,f,"[tensor(0.0009), tensor(0.0009), tensor(0.0009...",6.645313
1,3f_1612.wav,3f,3,f,"[tensor(-3.0518e-05), tensor(-3.0518e-05), ten...",3.232000
2,3f_1817.wav,3f,3,f,"[tensor(0.0008), tensor(0.0007), tensor(0.0009...",2.197312
3,3m_8446.wav,3m,3,m,"[tensor(0.0005), tensor(0.0005), tensor(0.0004...",1.408000
4,3m_9330.wav,3m,3,m,"[tensor(0.0002), tensor(0.0002), tensor(0.0002...",3.104000
...,...,...,...,...,...,...
3161,4m_6694.wav,4m,4,m,"[tensor(0.0002), tensor(0.0003), tensor(0.0002...",2.752000
3162,3f_1430.wav,3f,3,f,"[tensor(-0.0003), tensor(-0.0003), tensor(-0.0...",4.032000
3163,5f_8763.wav,5f,5,f,"[tensor(-3.0518e-05), tensor(-3.0518e-05), ten...",5.376000
3164,4m_9646.wav,4m,4,m,"[tensor(0.0003), tensor(0.0004), tensor(0.0003...",6.122625


## Loop data

In [ ]:
import math
target_duration = math.ceil(df_train_val.length.max()) # 13s
sr = list(sample_rates)[0] # 16kHz
for i in range(len(df_train_val.audio)):
    df_train_val.audio[i] = convenience.loop_audio(df_train_val.audio[i], df_train_val.length[i], target_duration, sr)

df_train_val

In [ ]:
# import math

# target_duration = math.ceil(df_train_val.length.max())
# sr = list(sample_rates)[0]

# df_train_val, _ = convenience.loop_audio_df(df_train_val, target_duration, sr)
# df_train_val.head()

## Normalize data

In [ ]:
import torch
df_train_val['audio_norm'] = df_train_val.looped_audio/[torch.max(torch.abs(df_train_val.looped_audio[i])) for i in range(len(df_train_val.looped_audio))]
df_train_val.head()

In [ ]:
maxs = [torch.max(torch.abs(df_train_val.audio_norm[i])) for i in range(len(df_train_val.audio_norm))]
mins = [torch.min(torch.abs(df_train_val.audio_norm[i])) for i in range(len(df_train_val.audio_norm))]
means = [torch.sum(torch.abs(df_train_val.audio_norm[i]))/len(df_train_val.audio_norm[i]) for i in range(len(df_train_val.audio_norm))]

print("Maximums")
print("min:\t",min(maxs))
print("max:\t",max(maxs))
print("mean:\t",sum(maxs)/len(maxs))

print("\nMinimums")
print("min:\t",min(mins))
print("max:\t",max(mins))
print("mean:\t",sum(mins)/len(mins))

print("\nMeans")
print("min:\t",min(means))
print("max:\t",max(means))
print("mean:\t",sum(means)/len(means))


## Split data

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df_train_val, test_size=0.2, random_state=42, stratify=df_train_val.stratify)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt

plt.title("Distribution of data")
plt.hist(df_train.sort_values('stratify').stratify)
plt.hist(df_val.sort_values('stratify').stratify)
plt.legend(("Train set", "Validation set"))
plt.xlabel("Combined labels")
plt.ylabel("Number of samples")

## Turn audio into spectrograms

In [ ]:
display(df_train.head())

In [ ]:
import librosa
import numpy as np
import os

if (not os.path.isdir("preprocessed")): os.mkdir("preprocessed")
if (not os.path.isdir("preprocessed/train")): os.mkdir("preprocessed/train")
if (not os.path.isdir("preprocessed/train/specs")): os.mkdir("preprocessed/train/specs")
if (not os.path.isdir("preprocessed/val")): os.mkdir("preprocessed/val")
if (not os.path.isdir("preprocessed/val/specs")): os.mkdir("preprocessed/val/specs")

labels_csv_out = ""

for i in range(len(df_train.looped_audio)):
    S = librosa.feature.melspectrogram(y=df_train.looped_audio[i].numpy(), sr=16000)
    labels_csv_out += ",".join([f"{df_train.file_name[i][:-4]}.npy", str(df_train.accent[i]),"\n"])
    np.save(f'preprocessed/train/specs/{df_train.file_name[i][:-4]}.npy', S)

with open("preprocessed/train/labels.csv", "w+") as f:
    f.write(labels_csv_out)

labels_csv_out = ""

for i in range(len(df_val.looped_audio)):
    S = librosa.feature.melspectrogram(y=df_val.looped_audio[i].numpy(), sr=16000)
    labels_csv_out += ",".join([f"{df_val.file_name[i][:-4]}.npy", str(df_val.accent[i]), "\n"])
    np.save(f'preprocessed/val/specs/{df_val.file_name[i][:-4]}.npy', S)


with open("preprocessed/val/labels.csv", "w+") as f:
    f.write(labels_csv_out)

In [ ]:
librosa.feature.melspectrogram(y=df_train_val.looped_audio[0].numpy(), sr=16000).shape

In [ ]:
# # Sanity check
import matplotlib.pyplot as plt
import os
S = np.load(f"preprocessed/train/specs/{os.listdir('preprocessed/train/specs')[1]}")
fig, ax = plt.subplots()
S_dB = librosa.power_to_db(S, ref=np.max)
img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=16000, ax=ax)

fig.colorbar(img, ax=ax, format='%+2.0f dB')

ax.set(title='Mel-frequency spectrogram')

## Process test data

In [ ]:
# Pre-process test set
import convenience
import torch
df_test, sample_rates = convenience.load_test()
display(df_test.head())

target_duration = 13
sr = 16000

df_test, _ = convenience.loop_audio_df(df_test, target_duration=target_duration, sr=sr)
display(df_test.head())

df_test['audio_norm'] = df_test.looped_audio/[torch.max(torch.abs(df_test.looped_audio[i])) for i in range(len(df_test.looped_audio))]
display(df_test.head())


In [ ]:
# Export test set
import librosa
import numpy as np
import os

if (not os.path.isdir("preprocessed")): os.mkdir("preprocessed")
if (not os.path.isdir("preprocessed/test")): os.mkdir("preprocessed/test")
if (not os.path.isdir("preprocessed/test/specs")): os.mkdir("preprocessed/test/specs")


labels_csv_out = "Id,label\n"

for i in range(len(df_test.looped_audio)):
    S = librosa.feature.melspectrogram(y=df_test.looped_audio[i].numpy(), sr=16000)
    labels_csv_out += ",".join([f"{df_test.id[i]}","\n"])
    np.save(f'preprocessed/test/specs/{df_test.file_name[i][:-4]}.npy', S)

with open("preprocessed/test/labels.csv", "w+") as f:
    f.write(labels_csv_out)